In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [4]:
!ls

datalab  drive


In [0]:
import os
os.chdir('drive/nnfl/NNFL')

In [6]:
import numpy as np
import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,Dropout
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [7]:
import h5py
hf = h5py.File('data_x.h5','r')
x = hf.get('dataset_1')
y = hf.get('dataset_2')
print(x.shape,y.shape)

(2997, 224, 224, 3) (2997, 12, 1)


In [8]:
y.shape

(2997, 12, 1)

I will resolve the issue with shuffling later

In [0]:
p=np.random.permutation(x.shape[0])
x = np.asarray(x)
y = np.asarray(y)
y = y.reshape((2997,12))
x = x[p]
y = y[p]

x_train = x[:2698]
y_train = y[:2698]
x_test = x[2698:]
y_test = y[2698:]

In [0]:
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters= F1,kernel_size=(1, 1), strides = (s,s),name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters= F2,kernel_size=(f, f), strides = (1,1),padding='same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters= F3,kernel_size= (1, 1), strides = (1,1), name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters= F3,kernel_size= (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [0]:
# GRADED FUNCTION: identity_block

def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    X = Dropout(0.5)(X)
    
    ### END CODE HERE ###
    
    return X

In [0]:
# GRADED FUNCTION: ResNet50

def ResNet50(input_shape = (224, 224, 3), classes = 12):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    #X = Dropout(0.5)(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    
    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2),name='avg_pool')(X)
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [0]:
model = ResNet50(input_shape = (224,224,3), classes = 12)

In [14]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [1]:
!ls

datalab


for 105 epochs without dropout

In [0]:
model = keras.models.load_model('resnet50.h5')

In [0]:
model.summary()

In [41]:
model.fit(x_train, y_train,validation_data=(x_test,y_test), epochs = 15, batch_size = 64)

Train on 2698 samples, validate on 299 samples
Epoch 1/15
2698/2698 [==============================] - 71s 26ms/step - loss: 0.5253 - acc: 0.8195 - val_loss: 1.9258 - val_acc: 0.4548
Epoch 2/15
2698/2698 [==============================] - 71s 26ms/step - loss: 0.5134 - acc: 0.8362 - val_loss: 2.0528 - val_acc: 0.4415
Epoch 3/15
2698/2698 [==============================] - 71s 26ms/step - loss: 0.7189 - acc: 0.7576 - val_loss: 1.8626 - val_acc: 0.4983
Epoch 4/15
2698/2698 [==============================] - 71s 26ms/step - loss: 0.5236 - acc: 0.8239 - val_loss: 2.0041 - val_acc: 0.4548
Epoch 5/15
2698/2698 [==============================] - 71s 26ms/step - loss: 0.5476 - acc: 0.8225 - val_loss: 2.2479 - val_acc: 0.4181
Epoch 6/15
 640/2698 [======>.......................] - ETA: 51s - loss: 0.4532 - acc: 0.8547

2698/2698 [==============================] - 71s 26ms/step - loss: 0.4456 - acc: 0.8554 - val_loss: 2.1818 - val_acc: 0.4047
Epoch 7/15
2698/2698 [==============================] - 71s 26ms/step - loss: 0.4025 - acc: 0.8666 - val_loss: 2.2696 - val_acc: 0.4247
Epoch 8/15
2698/2698 [==============================] - 71s 26ms/step - loss: 0.4853 - acc: 0.8480 - val_loss: 2.2326 - val_acc: 0.4348
Epoch 9/15
2698/2698 [==============================] - 71s 26ms/step - loss: 0.4503 - acc: 0.8603 - val_loss: 2.3425 - val_acc: 0.3880
Epoch 10/15
2698/2698 [==============================] - 71s 26ms/step - loss: 0.3802 - acc: 0.8744 - val_loss: 2.3904 - val_acc: 0.4281
Epoch 11/15
1856/2698 [===================>..........] - ETA: 21s - loss: 0.3795 - acc: 0.8863

2698/2698 [==============================] - 71s 26ms/step - loss: 0.3793 - acc: 0.8855 - val_loss: 2.2607 - val_acc: 0.4448
Epoch 12/15
2698/2698 [==============================] - 71s 26ms/step - loss: 0.4158 - acc: 0.8692 - val_loss: 2.5272 - val_acc: 0.4214
Epoch 13/15
2698/2698 [==============================] - 71s 26ms/step - loss: 0.3896 - acc: 0.8840 - val_loss: 2.4682 - val_acc: 0.4348
Epoch 14/15
2698/2698 [==============================] - 71s 26ms/step - loss: 0.3694 - acc: 0.8818 - val_loss: 2.4237 - val_acc: 0.4381
Epoch 15/15
2698/2698 [==============================] - 71s 26ms/step - loss: 0.3526 - acc: 0.8870 - val_loss: 2.4830 - val_acc: 0.4114


In [0]:
model.save('resnet50.h5')

with droput 135 epochs

In [0]:
model = keras.models.load_model('resnet50+dropout.h5')

In [0]:
model.summary()

In [35]:
model.fit(x_train, y_train,validation_data=(x_test,y_test), epochs = 15, batch_size = 64)

Train on 2698 samples, validate on 299 samples
Epoch 1/15
2698/2698 [==============================] - 78s 29ms/step - loss: 1.6261 - acc: 0.4411 - val_loss: 2.1863 - val_acc: 0.2776
Epoch 2/15
2698/2698 [==============================] - 78s 29ms/step - loss: 1.6496 - acc: 0.4359 - val_loss: 2.0794 - val_acc: 0.2943
Epoch 3/15
2698/2698 [==============================] - 78s 29ms/step - loss: 1.6000 - acc: 0.4485 - val_loss: 2.2749 - val_acc: 0.2943
Epoch 4/15
2698/2698 [==============================] - 78s 29ms/step - loss: 1.6505 - acc: 0.4403 - val_loss: 2.1260 - val_acc: 0.3144
Epoch 5/15
2698/2698 [==============================] - 78s 29ms/step - loss: 1.6123 - acc: 0.4451 - val_loss: 2.1587 - val_acc: 0.3043
Epoch 6/15
 640/2698 [======>.......................] - ETA: 57s - loss: 1.5544 - acc: 0.4422

2698/2698 [==============================] - 78s 29ms/step - loss: 1.5803 - acc: 0.4533 - val_loss: 2.1237 - val_acc: 0.3077
Epoch 7/15
2698/2698 [==============================] - 78s 29ms/step - loss: 1.5992 - acc: 0.4518 - val_loss: 2.1869 - val_acc: 0.3344
Epoch 8/15
2698/2698 [==============================] - 78s 29ms/step - loss: 1.5818 - acc: 0.4489 - val_loss: 2.1737 - val_acc: 0.3077
Epoch 9/15
2698/2698 [==============================] - 78s 29ms/step - loss: 1.5698 - acc: 0.4503 - val_loss: 2.3083 - val_acc: 0.2943
Epoch 10/15
2698/2698 [==============================] - 79s 29ms/step - loss: 1.5656 - acc: 0.4640 - val_loss: 2.1083 - val_acc: 0.2943
Epoch 11/15
1856/2698 [===================>..........] - ETA: 23s - loss: 1.5770 - acc: 0.4542

2698/2698 [==============================] - 79s 29ms/step - loss: 1.5596 - acc: 0.4577 - val_loss: 2.2804 - val_acc: 0.3043
Epoch 12/15
2698/2698 [==============================] - 78s 29ms/step - loss: 1.5579 - acc: 0.4670 - val_loss: 2.2594 - val_acc: 0.2977
Epoch 13/15
2698/2698 [==============================] - 78s 29ms/step - loss: 1.5119 - acc: 0.4800 - val_loss: 2.1445 - val_acc: 0.3211
Epoch 14/15
2698/2698 [==============================] - 78s 29ms/step - loss: 1.5064 - acc: 0.4785 - val_loss: 2.1147 - val_acc: 0.3344
Epoch 15/15
2698/2698 [==============================] - 78s 29ms/step - loss: 1.5173 - acc: 0.4870 - val_loss: 2.0515 - val_acc: 0.3545


In [0]:
model.save('resnet50+dropout.h5')